In [1]:
cd ..

C:\Users\johan\github\designresolver


In [2]:
from polycasebuilder import generate_random_polynomials, get_arg_mapping, directed_poly_executables, eqv_to_edges_tree
from testproblems import generate_random_prob
from graphutils import flat_graph_formulation, draw_graph_graphical_props, intermediary_variables, edges_to_Ein_Eout, Node, VAR, COMP, SOLVER, solver_children
from nestedgraph import build_typedgraph
from operators import sort_scc, reorder_merge_solve
from workflow import get_f, generate_workflow, implicit_comp_name
from assembly import build_archi
from executeformulations import extractvals, generate_x0, perturb_inputs, run_and_save_archi
import pandas as pd
from representations import bipartite_repr
from inputresolver import resolve,invert_edges,getallvars,direct_eqs

In [3]:
import openmdao.api as om

In [185]:
n_eqs = 50
n_vars = 60
seed = 2#8 is triang#seed 10 is nice 42
sparsity = 1.7#0.8 1.1 #1.7 1.3
eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed, sparsity)
polynomials, var_mapping, edges, tree, components = generate_random_polynomials(eqv, dout, n_eqs)
symb_mapping = get_arg_mapping(var_mapping, symbol_map=True)

In [184]:
# from utils import normalize_name
# inputs = {'x_11', 'x_23', 'x_25', 'x_29', 'x_32', 'x_33', 'x_45', 'x_48', 'x_56'}
# print('@var {}'.format(", ".join(['x_{}'.format(vr) for vr in range(n_vars)])))
# print('@var λ[1:{}];'.format(n_eqs))
# for key, var in polynomials.items():
#     print('h{}={};'.format(key, normalize_name(str(var), keep_underscore=True)))
# print('@var u[1:{}]'.format(len(inputs)))
# for idx,key in enumerate(inputs):
#     print('h{}={}-u{}'.format(idx+len(polynomials), key, idx))
# print('vars=[{}]'.format(", ".join(['x_{}'.format(vr) for vr in range(n_vars)])))
# #print('dfdx=[{}]'.format(", ".join([str(0) if vr not in [16, 22, 21] else 'x_{}'.format(vr) for vr in vrs])))
# print('h=[{}];'.format(", ".join(['h{}'.format(eq) for eq in polynomials.keys()])))

In [186]:
from graphutils import all_variables
from compute import Var
import numpy as np
output_set = dout
output_set = {key:var-n_eqs for key,var in output_set.items()}
edges, tree= eqv_to_edges_tree(eqv, output_set, n_eqs)
Ein, Eout, _ = edges
all_vars = all_variables(Ein,Eout)
var_mapping = {idx: (Var(str(Node(idx,VAR)), varid=idx), str(Node(idx,VAR))) for idx in all_vars}
rng = np.random.default_rng(12345)
arg_mapping = get_arg_mapping(var_mapping)

Optimize/presolve structure

In [187]:
eqs=direct_eqs(eqv, dout)
_, edges_original = bipartite_repr(eqv)
avrs = getallvars(eqs, sympy=False)
eqns = eqs.keys()
graph_edges = invert_edges(edges_original)

In [188]:
output = resolve(eqns, avrs, graph_edges, 50)

In [189]:
soutput = sorted(output, key=lambda x: x['CLEN'])

In [190]:
soutput[0]['CLEN']

[36.0]

In [158]:
out_opt = {eq:var-n_eqs for eq,var in soutput[0]['SOL']}

In [159]:
edges_opt, tree_opt = eqv_to_edges_tree(eqv, out_opt, n_eqs)

In [160]:
new_components = directed_poly_executables(var_mapping, polynomials, out_opt)

In [161]:
components += new_components

In [176]:
#components

# MDF formulations

In [137]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)

## Preprocessed

In [138]:
from runpipeline import get_solver_implicit_system

In [168]:
G2 = flat_graph_formulation(*edges_opt)
merge_order2 = sort_scc(G2)
nedges_opt, ntree_opt = reorder_merge_solve(edges_opt, tree_opt, merge_order2, 1, True)
lookup_f = get_f(components, nedges_opt)
workflow2 = generate_workflow(lookup_f, nedges_opt, ntree_opt, {2:{'maxiter':100}})

In [169]:
prob, mdao_in, groups = build_archi(nedges_opt, ntree_opt, workflow2)

In [141]:
solver_idx = 2
impl_system = get_solver_implicit_system(groups, ntree_opt, solver_idx) # we use this for counting

In [142]:
rng = np.random.default_rng(1234)

In [143]:
#xref=optres
scc_guess_vars = [str(Node(vr, VAR)) for vr in solver_children(ntree_opt[2], solver_idx)]
#scc_guess_vars = []
root_rand_range = (0,0)
solver_rand_range = (0,0.01)
print(scc_guess_vars)
x0_in = perturb_inputs(mdao_in, root_rand_range, scc_guess_vars, solver_rand_range, rng=rng)

['x_6', 'x_3']


In [144]:
#x0_in = {'x_1':0.279144, 'x_5':-0.278548, 'x_7':13.190141}

In [145]:
for key,val in x0_in.items():
    prob.set_val(key,val)
prob.run_model()


===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 1 iterations
NL: Newton Converged in 0 iterations


In [146]:
prob.model.list_outputs();

7 Explicit Output(s) in 'model'

varname   val          
--------  -------------
inp
  x_0     [0.1]        
  x_10    [0.1]        
  x_4     [0.1]        
s_1
  f_6
    x_7   [-1.14285714]
  f_0
    x_9   [15.]        
  f_2
    x_12  [5.55555556] 
  f_3
    x_1   [-12.5]      


2 Implicit Output(s) in 'model'

varname        val          
-------------  -------------
s_1
  s_2
    res_f9_f5
      x_6      [17.89285714]
      x_3      [2.49107143] 




# Build workflow

In [26]:
def get_solver_implicit_system(groups, tree, solver_idx):
    s = groups[str(Node(solver_idx, SOLVER))]
    name = implicit_comp_name((Node(idx, COMP) for idx in solver_children(tree[0], solver_idx)))
    return getattr(s, name)

In [17]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)
lookup_f = get_f(components, nedges)
workflow = generate_workflow(lookup_f, nedges, ntree, {2: {'maxiter':100}})
prob, mdao_in, groups = build_archi(nedges, ntree, workflow)
solver_idx = 2
impl_system = get_solver_implicit_system(groups, ntree, solver_idx) # we use this for counting
scc_guess_vars = [str(Node(vr, VAR)) for vr in solver_children(ntree[2], solver_idx)]

In [18]:
scc_guess_vars

['x_3', 'x_7', 'x_1', 'x_10', 'x_2', 'x_9', 'x_5', 'x_4', 'x_8', 'x_0']

# Automate runflow

In [163]:
def run_full(edges, tree, components, xref=None, solver_rand_range=(0,1)):
    optres_all = []
    solver_idx = 2
    G = flat_graph_formulation(*edges)
    merge_order = sort_scc(G)
    nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)
    print(ntree)
    lookup_f = get_f(components, nedges)
    workflow = generate_workflow(lookup_f, nedges, ntree, {2:{'maxiter':100}})
    prob, mdao_in, groups = build_archi(nedges, ntree, workflow)
    print(mdao_in)
    impl_system = get_solver_implicit_system(groups, ntree, solver_idx) # we use this for counting
    scc_guess_vars = [str(Node(vr, VAR)) for vr in solver_children(ntree[2], solver_idx)]
    rng = np.random.default_rng(1234)
    root_rand_range = (0,0)
    print(scc_guess_vars)
    x0_in = perturb_inputs(mdao_in, root_rand_range, scc_guess_vars, solver_rand_range, xref, rng)
    print(x0_in)
    optres, optres_save = run_and_save_archi(prob, x0_in, symb_mapping)
    optres_save.update({'count': impl_system.iter_count_apply}) 
    optres_all.append(optres_save)
    return optres, x0_in, prob,workflow

In [164]:
optres, x0_in, prob,_ = run_full(edges, tree, components)

(OrderedDict([(13, 1), (30, 1), (17, 1), (28, 1), (46, 1), (14, 1), (20, 1), (48, 1), (23, 2), (12, 2), (45, 2), (34, 2), (3, 2), (47, 2), (36, 2), (5, 2), (25, 2), (16, 2), (49, 2), (18, 2), (40, 2), (29, 2), (7, 2), (31, 2), (42, 2), (11, 2), (0, 2), (44, 2), (24, 2), (35, 2), (26, 2), (15, 2), (6, 2), (39, 2), (37, 2), (8, 2), (19, 2), (41, 2), (10, 2), (32, 2), (1, 2), (21, 2), (9, 1), (2, 1), (4, 3), (27, 3), (33, 1), (38, 1), (22, 1), (43, 1)]), {2: 1, 3: 1}, {23: 2, 6: 2, 40: 2, 38: 2, 10: 2, 15: 2, 50: 2, 20: 2, 19: 2, 36: 2, 4: 2, 25: 2, 13: 2, 42: 2, 47: 2, 35: 2, 29: 2, 43: 2, 34: 2, 57: 2, 22: 2, 31: 2, 44: 2, 32: 2, 3: 2, 58: 2, 52: 2, 59: 2, 26: 2, 12: 2, 33: 2, 5: 2, 2: 2, 1: 2, 21: 3, 46: 3})
{'x_45', 'x_0', 'x_48', 'x_39', 'x_51', 'x_11', 'x_18', 'x_56', 'x_41'}
['x_23', 'x_6', 'x_40', 'x_38', 'x_10', 'x_15', 'x_50', 'x_20', 'x_19', 'x_36', 'x_4', 'x_25', 'x_13', 'x_42', 'x_47', 'x_35', 'x_29', 'x_43', 'x_34', 'x_57', 'x_22', 'x_31', 'x_44', 'x_32', 'x_3', 'x_58', 'x_5

C:\Users\johan\miniconda3\envs\mdo\lib\site-packages\autograd\tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2': residuals contain 'inf' or 'NaN' after 36 iterations.
+  
+  =======
+  s_1.s_3
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_3': residuals contain 'inf' or 'NaN' after 0 iterations.
NL: NewtonSolver 'NL: Newton' on system 's_1': residuals contain 'inf' or 'NaN' after 1 iterations.


<lambdifygenerated-303>:2: RuntimeWarning: overflow encountered in multiply
  return -0.7*x_12*x_31*x_5 + 0.1
C:\Users\johan\miniconda3\envs\mdo\lib\site-packages\openmdao\solvers\solver.py:599: RuntimeWarning: invalid value encountered in double_scalars
  self._mpi_print(self._iter_count, norm, norm / norm0)
C:\Users\johan\miniconda3\envs\mdo\lib\site-packages\openmdao\solvers\solver.py:606: RuntimeWarning: invalid value encountered in double_scalars
  while self._iter_count < maxiter and norm > atol and norm / norm0 > rtol and not stalled:


# Checking results

In [22]:
outs = extractvals(prob, get_arg_mapping(var_mapping, symbol_map=True).values())

In [23]:
comps_res = [comp for comp in components if comp.outputs == (None,)]

In [24]:
[elt.evaldict(outs) for elt in comps_res]

[1.1102230246251565e-16,
 2.220446049250313e-16,
 -1.6653345369377348e-16,
 -1.1102230246251565e-16,
 -3.3306690738754696e-16,
 -5.551115123125783e-17,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 0.0]

In [25]:
x0_in

{'x_12': 0.1,
 'x_11': 0.1,
 'x_6': 0.1,
 'x_4': -0.018091232966642806,
 'x_8': 0.34176629325278507,
 'x_0': -0.8640792451783763,
 'x_3': 0.36364980427509375,
 'x_7': -0.5098708094225075,
 'x_1': 0.9636212965699608,
 'x_10': 0.774881313349695,
 'x_2': -0.5598743479594311,
 'x_9': 0.32275365813138723,
 'x_5': 0.2720661846611385}

In [26]:
optres

{'x_0': -7.14496195261011,
 'x_1': 0.27914388981534266,
 'x_2': 29.62989894048818,
 'x_3': 0.017925643076510026,
 'x_4': 30.117299373280968,
 'x_5': -0.2785476176509625,
 'x_6': 0.1,
 'x_7': 13.19014100182846,
 'x_8': 0.023874438126960634,
 'x_9': 0.033073507098491065,
 'x_10': 2.0217120243801827,
 'x_11': 0.1,
 'x_12': 0.1}

In [62]:
import openmdao.api as om
om.n2(prob2)

In [166]:
optres2, x0_in2, prob2, wf = run_full(edges_opt, tree_opt, components, solver_rand_range=(0,1))

(OrderedDict([(13, 1), (17, 1), (28, 1), (30, 1), (1, 2), (36, 2), (39, 1), (14, 1), (20, 1), (34, 1), (18, 1), (42, 1), (8, 1), (24, 1), (16, 1), (15, 1), (31, 1), (23, 3), (47, 3), (38, 1), (21, 1), (22, 1), (29, 4), (35, 4), (19, 4), (41, 4), (5, 4), (32, 4), (6, 4), (25, 4), (37, 4), (0, 4), (45, 1), (2, 1), (10, 1), (43, 1), (49, 1), (46, 1), (48, 1), (11, 1), (3, 1), (12, 5), (7, 5), (40, 5), (26, 5), (4, 6), (27, 6), (9, 1), (33, 1), (44, 1)]), {2: 1, 3: 1, 4: 1, 5: 1, 6: 1}, {2: 2, 50: 2, 15: 3, 20: 3, 42: 4, 31: 4, 26: 4, 12: 4, 41: 4, 5: 4, 3: 4, 19: 4, 52: 4, 34: 4, 6: 5, 51: 5, 47: 5, 44: 5, 46: 6, 21: 6})
{'x_45', 'x_48', 'x_29', 'x_11', 'x_25', 'x_23', 'x_33', 'x_56', 'x_32'}
['x_2', 'x_50']
{'x_45': 0.1, 'x_48': 0.1, 'x_29': 0.1, 'x_11': 0.1, 'x_25': 0.1, 'x_23': 0.1, 'x_33': 0.1, 'x_56': 0.1, 'x_32': 0.1, 'x_2': 0.774881313349695, 'x_50': -0.5598743479594311}

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to

<lambdifygenerated-415>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return -1.5/(x_15*x_57)
C:\Users\johan\miniconda3\envs\mdo\lib\site-packages\openmdao\vectors\default_vector.py:423: RuntimeWarning: invalid value encountered in add
  data[idxs] += val


In [59]:
wf

[(SOLVE, None, 's_1', {}),
 (EXPL,
  's_1',
  'f_1',
  ['x_1', 'x_5'],
  ['x_0'],
  <bound method Component.evaldict of ((1, 5), 1, (0,), '0.555555555555556/(x_1*x_5)')>,
  <bound method Component.graddict of ((1, 5), 1, (0,), '0.555555555555556/(x_1*x_5)')>,
  False),
 (EXPL,
  's_1',
  'f_7',
  ['x_0', 'x_1', 'x_5'],
  ['x_9'],
  <bound method Component.evaldict of ((0, 1, 5), 7, (9,), '-(1.66666666666667*x_0*x_1 + 3.33333333333333)/x_5')>,
  <bound method Component.graddict of ((0, 1, 5), 7, (9,), '-(1.66666666666667*x_0*x_1 + 3.33333333333333)/x_5')>,
  False),
 (EXPL,
  's_1',
  'f_8',
  ['x_7', 'x_0'],
  ['x_8'],
  <bound method Component.evaldict of ((7, 0), 8, (8,), '-2.25/(x_0*x_7)')>,
  <bound method Component.graddict of ((7, 0), 8, (8,), '-2.25/(x_0*x_7)')>,
  False),
 (EXPL,
  's_1',
  'f_3',
  ['x_8', 'x_0', 'x_9'],
  ['x_4'],
  <bound method Component.evaldict of ((8, 0, 9), 3, (4,), '(x_0*x_8 + 1.16666666666667)/x_9')>,
  <bound method Component.graddict of ((8, 0, 9), 

In [64]:
def run_full(edges, tree, components, x0_in=None):
    optres_all = []
    solver_idx = 2
    G = flat_graph_formulation(*edges)
    merge_order = sort_scc(G)
    nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)
    lookup_f = get_f(components, nedges)
    workflow = generate_workflow(lookup_f, nedges, ntree, {2:{'maxiter':100}})
    prob, mdao_in, groups = build_archi(nedges, ntree, workflow)
    impl_system = get_solver_implicit_system(groups, ntree, solver_idx) # we use this for counting
    scc_guess_vars = [str(Node(vr, VAR)) for vr in solver_children(ntree[2], solver_idx)]
    prob.setup();
    rand_range = (0,1)
    if not x0_in:
        x0_in = generate_x0(mdao_in, rand_range=(0,0))
    for i in range(5):
        x0 = dict(x0_in)
        #x0_scc = generate_x0(scc_guess_vars, rand_range=rand_range)
        #x0.update(x0_scc) # this could override x0 ins from scc but thats ok
        #print(x0)
        optres, optres_save = run_and_save_archi(prob, x0, symb_mapping)
        optres_save.update({'count': impl_system.iter_count_apply}) 
        optres_all.append(optres_save)
    return optres_all

In [62]:
pd.DataFrame.from_dict(results_default_case)

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,count
0,-13.226201,0.150677,5.475009,-0.536488,33.149296,-0.278770,0.1,-9.191052,-0.018509,0.042579,-2.901373,0.1,0.1,262
1,-13.226205,0.150677,5.475003,-0.536488,33.149298,-0.278770,0.1,-9.191051,-0.018509,0.042579,-2.901373,0.1,0.1,181
2,46.392218,-0.000417,-0.454694,0.516928,29.078760,-28.693481,0.1,-1.032716,0.046963,0.115046,-25.821888,0.1,0.1,119
3,0.113836,-16.977890,13.821738,-0.561735,2.305145,-0.287451,0.1,8.422801,-2.346637,0.390229,3.166009,0.1,0.1,125
4,0.113836,-16.977890,13.821738,-0.561735,2.305145,-0.287451,0.1,8.422801,-2.346637,0.390229,3.166009,0.1,0.1,34


In [61]:
results_default_case = run_full(edges, tree, components)
x0 = {key:var for key,var in results_default_case[0].items() if key != 'count'}

{'x_12': 0.1, 'x_11': 0.1, 'x_6': 0.1, 'x_10': -0.3866379968583621, 'x_2': 0.2877212286612516, 'x_9': 0.7494588832440994, 'x_5': 0.5413092228959531, 'x_4': -0.6643726086611824, 'x_8': 0.3740864619922246, 'x_0': 0.2190953574782604, 'x_3': 0.7287879088794833, 'x_7': 0.5921359986061542, 'x_1': -0.16581911753550724}

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 56 iterations
NL: Newton Converged in 2 iterations
{'x_12': 0.1, 'x_11': 0.1, 'x_6': 0.1, 'x_10': 0.006636966365663716, 'x_2': -0.2275639476887341, 'x_9': 0.9924110171617034, 'x_5': 1.063930529067942, 'x_4': 0.009451304462442939, 'x_8': 0.12430656162948697, 'x_0': 0.784915607421425, 'x_3': -0.6795145855555298, 'x_7': -0.46318750662850483, 'x_1': 0.101

In [65]:
results_opt_case = run_full(edges_opt, tree_opt, components, x0_in=x0)


===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Co

In [43]:
results_default_case[0]

{'x_0': 0.008958106779525476,
 'x_1': -13.145086033480357,
 'x_2': 0.7424238083735113,
 'x_3': 12.148139891511779,
 'x_4': -1.4592582797318507,
 'x_5': -4.717890176820716,
 'x_6': -10.79821037737607,
 'x_7': 1.0528907779309629,
 'x_8': -238.5519447614448,
 'x_9': 0.6649317262384284,
 'x_10': -0.14423932736560077,
 'x_11': -19.086109079379213,
 'x_12': -17.559078564803155,
 'count': 106}

In [41]:
pd.DataFrame.from_dict(results_default_case)

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,count
0,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,106
1,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2
2,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2
3,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2
4,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2


In [16]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)

In [17]:
merge_order

[{f_1, f_3, f_4, f_7, f_8, f_9}, {f_5}, {f_2}, {f_6}, {f_0}]

In [6]:
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)

In [7]:
lookup_f = get_f(components, nedges)

In [8]:
workflow = generate_workflow(lookup_f, nedges, ntree, {2:{'maxiter':100}})

In [9]:
prob, mdao_in, groups = build_archi(nedges, ntree, workflow)

In [10]:
prob.setup();

In [11]:
symb_mapping = get_arg_mapping(var_mapping, symbol_map=True)

In [12]:
scc_guess_vars = [str(Node(vr, VAR)) for vr in ntree[2].keys()]

In [43]:
x0_in = generate_x0(mdao_in, rand_range=(10,20))

In [48]:
rand_range = (100,200)
optres_all=[]
for i in range(5):
    x0_scc = generate_x0(scc_guess_vars, rand_range=rand_range)
    optres, optres_save = run_and_save_archi(prob, {**x0_in, **x0_scc}, symb_mapping)
    optres_save.update({'count': impl_system.iter_count_apply}) 
    optres_all.append(optres_save)
        #x0 = generate_x0(optres=optres, rand_range=rand_range)


===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 8 iterations
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 32 iterations
NL: Newton Converged in 0 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 38 iterations
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 26 iterations
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 50 iterations
NL: Newton Converged in 0 iterations
